## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
# Connect to DB with a CSV fallback
from Python_scripts import config
import pandas as pd
import platform
import os

# Try to connect to PostgreSQL using centralized config; fall back to CSV files in DATA_DIR.
try:
    conn = config.get_conn()
    cursor = conn.cursor()
    use_csv_fallback = False
    print("[INFO] Connected to Postgres")
except Exception as exc:
    print("[WARN] DB connection failed, falling back to CSVs:", exc)
    conn = None
    cursor = None
    use_csv_fallback = True
    data_dir = config.get_data_dir()
    print(f"[INFO] Using DATA_DIR = {data_dir}")

    # Load available dlc_table_*.csv files into a dict for later fallback lookups
    dlc_tables = {}
    if data_dir.exists():
        for p in data_dir.glob('dlc_table_*.csv'):
            try:
                dlc_tables[p.name] = pd.read_csv(p)
                print(f"[INFO] Loaded {p.name}")
            except Exception as e:
                print(f"[WARN] Couldn't load {p}: {e}")
    else:
        print(f"[WARN] DATA_DIR does not exist: {data_dir}")


## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list


In [ ]:
import importlib
import Python_scripts.Feature_functions.motion_features_per_minute
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.motion_features_per_minute)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.motion_features_per_minute import (
    compute_motion_features_per_minute, batch_compute_motion_features_per_minute
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


## Fetch id list

In [ ]:
task_name = "AllTask"
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=None, #['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly'],
    dose_mult=10,
    genotype="white",
    bad_ids=bad_id,
    csv_prefix="dlc_table",   # -> dlc_table_saline.csv, dlc_table_ghrelin.csv, ...
    min_trial_length=None     # or 600
)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"exc_id: {exc_id}\n")
print(f"inh_id: {inh_id}\n")


## Calculate velocity

### Batch call with different input parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
bodypart = 'Head'          # or 'Midback'
time_limit = None          # e.g., 120 for first 2 minutes; None for full trial
smooth = True              # IMPORTANT: set True, otherwise 'window' has no effect
window_range = range(5, 26, 5)  # 11,15,19,23,27

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    # "Ghrelin": ghrelin_id,
    "Inhibitory": inh_id,
    "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_{bodypart}_velocity_window_sweep.pdf"
with PdfPages(outfile) as pdf:
    for window_size in window_range:
        print(f"[INFO] Analyzing window_size = {window_size}")

        # Compute all groups for this window size
        frames = []
        for label, ids in group_specs.items():
            df = batch_compute_motion_features_per_minute(
                conn, ids,
                bodypart=bodypart,
                time_limit=time_limit,
                smooth=smooth,
                window=window_size,
                # min_duration_s=5.0,  # keep default unless you want to drop very short trials
            )
            # Keep what we need; add group label
            df = df[['trial_id', 'velocity_per_min']].copy().dropna()
            df['group'] = label
            frames.append(df)

        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue

        df_vel = pd.concat(frames, ignore_index=True)

        # Plot (your existing plotter)
        order = list(group_specs.keys())
        fig, ax = plot_groupwise_bar(
            df_vel,
            y='velocity_per_min',
            ylabel='Average speed (units/min)',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ranksums", "ttest", "anova")  # if supported
        )
        ax.set_title(f"{task_name} | {bodypart} | window={window_size}", pad=20)

        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")


### Single call with fixed hyper parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Configure once ---
bodypart = 'Head'
time_limit = None
smooth = False
window = 5  # smoothing window for compute_motion_features (if smooth=True)

# Put every group we *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,    # lists/arrays of trial IDs, defined earlier
    "Ghrelin": ghrelin_id,
    # "Inhibitory": inh_id,
    # "Excitatory": exc_id,
}

# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

# Compute all groups
frames = []
for label, ids in group_specs.items():
    df = batch_compute_motion_features_per_minute(
        conn, ids,
        bodypart=bodypart,
        time_limit=time_limit,
        smooth=smooth,
        window=window,
        # min_duration_s=5.0,
    )
    # Keep just what we need for plotting; add group label
    df = df[['trial_id', 'velocity_per_min']].copy()
    df['group'] = label
    frames.append(df)

if not frames:
    raise ValueError("[WARN] No groups provided—nothing to plot.")

df_vel = pd.concat(frames, ignore_index=True)
df_vel = df_vel.dropna(subset=['velocity_per_min'])

# Plot
order = list(group_specs.keys())  # consistent ordering across pages
ax = plot_groupwise_bar(
    df_vel, y='velocity_per_min',
    ylabel='Average speed (units/min)',
    plot_type='bar', show_points=False,
    order=order, show_stats=True,
    tests_to_show=("ranksums","ttest")  # optional
)
fig = ax[0]

In [ ]:
print(ax[2])

In [ ]:
ax[1].set_title(f"{task_name} | {bodypart} | window={window}", pad=20)
# ax[1].set_ylim([0, 12])
fig.savefig(f"White_10X_{task_name}_velocity.pdf", dpi=300, bbox_inches='tight')

## Save Excel file for data submission

In [ ]:
# Step 1. Get the unique trial IDs from df_vel
trial_ids = df_vel['trial_id'].unique().tolist()

# Step 2. Query the database for task & modulation
query = """
SELECT id, task, modulation
FROM dlc_table
WHERE id = ANY(%s);
"""
meta_df = pd.read_sql_query(query, conn, params=(trial_ids,))

# Step 3. Merge with df_vel on id
df_out = df_vel.merge(meta_df, left_on='trial_id', right_on='id', how='left')

# Optional: drop redundant id column if you just want trial_id
df_out = df_out.drop(columns=['id'])

# Step 4. Save to Excel
df_out.to_excel("10X_White_Velocity.xlsx", index=False)
